<a href="https://colab.research.google.com/github/Afshan-Khalid/ProjectBlindAssist/blob/main/Done_dataDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
cd /content/gdrive/My Drive

/content/gdrive/My Drive


In [43]:
!mkdir OpenImages

mkdir: cannot create directory ‘OpenImages’: File exists


In [3]:
cd OpenImages

/content/gdrive/MyDrive/OpenImages


In [39]:
!pwd

/content/gdrive/My Drive/OpenImages


In [47]:
# Download required meta-files
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv

!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv

!wget https://storage.googleapis.com/openimages/2018_04/validation/validation-annotations-bbox.csv

!wget https://storage.googleapis.com/openimages/2018_04/test/test-annotations-bbox.csv

--2024-07-28 23:26:26--  https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.207, 142.250.99.207, 74.125.20.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11255 (11K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  10.99K  --.-KB/s    in 0.008s  

2024-07-28 23:26:26 (1.41 MB/s) - ‘class-descriptions-boxable.csv’ saved [11255/11255]

--2024-07-28 23:26:26--  https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.207, 142.250.99.207, 74.125.20.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194033454 (1.1G) [

In [40]:
!ls

class-descriptions-boxable.csv	test-annotations-bbox.csv   train.tfrecord
downloadOI.py			test.tfrecord		    validation
models				train			    validation-annotations-bbox.csv
test				train-annotations-bbox.csv  validation.tfrecord


In [6]:
!pip install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
readme-renderer 44.0 requires docutils>=0.21.2, but you have docutils 0.16 which is incompatible.


In [7]:
# This magic function is used to create the files.downloadOI.py is the file name
%%writefile downloadOI.py


#### Usage example: python3 downloadOI.py --classes 'Sink,Toilet' --mode test

import argparse
import csv
import subprocess
import os
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool as thread_pool

cpu_count = multiprocessing.cpu_count()

parser = argparse.ArgumentParser(description='Download Class specific images from OpenImagesV4')
parser.add_argument("--mode", help="Dataset category - train, validation or test", required=True)
parser.add_argument("--classes", help="Names of object classes to be downloaded", required=True)
parser.add_argument("--nthreads", help="Number of threads to use", required=False, type=int, default=cpu_count*2)
parser.add_argument("--occluded", help="Include occluded images", required=False, type=int, default=1)
parser.add_argument("--truncated", help="Include truncated images", required=False, type=int, default=1)
parser.add_argument("--groupOf", help="Include groupOf images", required=False, type=int, default=1)
parser.add_argument("--depiction", help="Include depiction images", required=False, type=int, default=1)
parser.add_argument("--inside", help="Include inside images", required=False, type=int, default=1)

args = parser.parse_args()

run_mode = args.mode
threads = args.nthreads
classes = args.classes.split(',')

# Single directory for all images
output_dir = run_mode
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open('./class-descriptions-boxable.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_list = {rows[1]:rows[0] for rows in reader}

subprocess.run(['rm', '-rf', output_dir])
subprocess.run(['mkdir', output_dir])

pool = thread_pool(threads)
commands = []
cnt = 0

# Initialize image count dictionary for each class
class_image_count = {cls: 0 for cls in classes}

for ind in range(len(classes)):
    class_name = classes[ind]
    print(f"Class {ind} : {class_name}")

    command = f"grep {dict_list[class_name.replace('_', ' ')]} ./{run_mode}-annotations-bbox.csv"
    class_annotations = subprocess.run(command.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')
    class_annotations = class_annotations.splitlines()

    for line in class_annotations:
        if class_image_count[class_name] >= 100:
            print(f"Reached 100 images for class {class_name}")
            break

        line_parts = line.split(',')

        # IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
        if (args.occluded == 0 and int(line_parts[8]) > 0):
            print(f"Skipped {line_parts[0]}")
            continue
        if (args.truncated == 0 and int(line_parts[9]) > 0):
            print(f"Skipped {line_parts[0]}")
            continue
        if (args.groupOf == 0 and int(line_parts[10]) > 0):
            print(f"Skipped {line_parts[0]}")
            continue
        if (args.depiction == 0 and int(line_parts[11]) > 0):
            print(f"Skipped {line_parts[0]}")
            continue
        if (args.inside == 0 and int(line_parts[12]) > 0):
            print(f"Skipped {line_parts[0]}")
            continue

        cnt += 1
        class_image_count[class_name] += 1

        command = f'aws s3 --no-sign-request --only-show-errors cp s3://open-images-dataset/{run_mode}/{line_parts[0]}.jpg {output_dir}/{line_parts[0]}.jpg'
        commands.append(command)

        with open(f'{output_dir}/{line_parts[0]}.txt', 'a') as f:
            f.write(','.join([class_name, line_parts[4], line_parts[5], line_parts[6], line_parts[7]]) + '\n')

    # Stop processing once 100 images have been collected for all classes
    if all(count >= 100 for count in class_image_count.values()):
        break

print(f"Annotation Count: {cnt}")
commands = list(set(commands))
print(f"Number of images to be downloaded: {len(commands)}")

list(tqdm(pool.imap(os.system, commands), total=len(commands)))

pool.close()
pool.join()


Overwriting downloadOI.py


In [8]:
# Download  images for test
!python3 downloadOI.py --classes "Stairs,Tree,House,Person,Cat,Dog,Traffic light,Box,Waste container" --mode test

# Download  images for train
!python3 downloadOI.py --classes "Stairs,Tree,House,Person,Cat,Dog,Traffic light,Box,Waste container" --mode train

# Download  images for validation
!python3 downloadOI.py --classes "Stairs,Tree,House,Person,Cat,Dog,Traffic light,Box,Waste container" --mode validation


Class 0 : Stairs
Reached 100 images for class Stairs
Class 1 : Tree
Reached 100 images for class Tree
Class 2 : House
Reached 100 images for class House
Class 3 : Person
Reached 100 images for class Person
Class 4 : Cat
Reached 100 images for class Cat
Class 5 : Dog
Reached 100 images for class Dog
Class 6 : Traffic light
Reached 100 images for class Traffic light
Class 7 : Box
Reached 100 images for class Box
Class 8 : Waste container
Reached 100 images for class Waste container
Annotation Count: 900
Number of images to be downloaded: 587
100% 587/587 [06:06<00:00,  1.60it/s]
Class 0 : Stairs
Reached 100 images for class Stairs
Class 1 : Tree
Reached 100 images for class Tree
Class 2 : House
Reached 100 images for class House
Class 3 : Person
Reached 100 images for class Person
Class 4 : Cat
Reached 100 images for class Cat
Class 5 : Dog
Reached 100 images for class Dog
Class 6 : Traffic light
Reached 100 images for class Traffic light
Class 7 : Box
Reached 100 images for class Box
Cl